In [ ]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='student02'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F

In [ ]:
conf = SparkConf().setAppName('AQE_Use_Broadcast_20240519')\
    .setMaster('yarn') \
    .set('spark.sql.adaptive.enabled', 'false')\
    .set('spark.sql.shuffle.partitions', '5')\

    # .set('spark.sql.autoBroadcastJoinThreshold', 10485760 )

    # .set('spark.sql.adaptive.coalescePartitions.initialPartitionNum', '250')\
    # .set('spark.default.parallelism', '4')\
    # .set('spark.dynamicAllocation.enabled', 'false')\
    # .set("spark.sql.sources.partitionOverwriteMode", "dynamic") 
 # Required for overwriting ONLY the required partitioned folders, and not the entire root folder

spark = SparkSession.builder.config(conf=conf) \
    .getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(40), sc.applicationId)
print("spark.sql.autoBroadcastJoinThreshold".ljust(40), spark.conf.get("spark.sql.autoBroadcastJoinThreshold"))

In [ ]:
import random
from datetime import date, timedelta
end = date.today()
start = end + timedelta(days=-300)
names = ['Bob', 'James', 'Marek', 'Johannes', 'Иванов', 'Смирнов', 'Кузнецов', 'Попов', 'Васильев', 'Петров', 'Соколов', 'Михайлов', 'Новиков', 'Федоров', 'Морозов', 'Волков', 'Алексеев', 'Лебедев', 'Семенов', 'Егоров', 'Павлов', 'Козлов', 'Степанов', 'Николаев']
countries = [('rus', 'Russia'),('bel', 'Belgium'), ('usa', 'United States'), 
             ('fra', 'France'), ('aus', 'Australia'), ('ita', 'Italy'), 
             ('swe', 'Sweden'), ('geo', 'Georgia'), ('pol', 'Poland'),
             ('che', 'Cheska'), ('slo', 'Slovenia'), ('ser', 'Serbia'),
             ('mon', 'Montenegro'), ('bul', 'Bulgaria'), ('rom', 'Romania'),
             ('fin', 'Finland'), ('ukr', 'Ukraine'), ('blr', 'Belorusia'),
             ('chi', 'China')]
df_countries = spark.createDataFrame(data=countries, schema=["country", "country_name",])
# save_path = "/user/student02/customers_partitioned_year_month"
sc.setJobDescription('MY DESCRIPTION JOB 1')

In [ ]:
length_customers = 1000000
customers = [None]*length_customers
for i in range(length_customers):
    random_date = start + (end - start) * random.random()

    if i % 2 == 0:
        country = countries[-1][0] # 50% take last - 'China'
    else:
        country = countries[random.randint(0, len(countries)-2)][0] # all without China
    
    customers[i] = (names[random.randint(0, len(names)-1)], country, random.randint(-1000000, 1000000), random_date)

df_customers = spark.createDataFrame(data=customers, schema=["name","country","balance","last_order_date"])

In [ ]:
df_customers.createOrReplaceTempView("customers")
df_countries.createOrReplaceTempView("countries")

In [ ]:
# df_joined = df_customers.join(df_countries, "country", "leftouter")
# df_to_explain = df_customers.join(df_countries.hint("broadcast"), 'country', 'leftouter')

# df_to_explain = df_customers.groupBy("country").agg(F.count("name"))
# sc.setJobDescription('MY JOB group by rollup(country)')
df_to_explain = spark.sql(" select name, country, country_name \
                 from customers left outer join countries using (country) \
                 CLUSTER BY name ")

                # order by country desc"
# df_to_explain = spark.sql(sql_clause).show()
# spark.sql("EXPLAIN " + sql_clause).show(60, False, False)
# .orderBy("name", "year", "month") #.show()
# df_grouped.rdd.getNumPartitions()
# df_to_explain.explain(True)
# df_to_explain = df_joined.groupBy('country_name').agg(F.count(F.lit("1")))
df_to_explain.withColumn('pid', F.spark_partition_id())\
   .groupBy(["name","pid"]).count().orderBy(["pid"]).collect()

In [ ]:
# df_to_explain.collect()
print(df_to_explain.rdd.getNumPartitions())

In [ ]:
--

In [ ]:
spark.stop()